## test-google-news-index

In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/apple/WRK/dcolinmorgan/dots'

In [2]:
from DOTS.scrape import get_OS_data, get_test_gnews
from DOTS.pull import pull_data, process_url
from DOTS.ingestion_utils import safe_iter_pull, iter_pull, reduce_newlines, scrape_selenium_headless

In [4]:
from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
os_url = os.getenv('OS_TOKEN')
openai_KEY = os.getenv('OPENAI_API_KEY')

lobstr_key = os.getenv('LOBSTR_KEY')

In [5]:
response = get_test_gnews(1000)
hits = response["hits"]["hits"]
articles = pull_data(hits)
indices = [i for i, x in enumerate(articles) if len(str(x)) < 50]
print(len(indices))
# for i in indices:
#     url = hits[i]['_source']['metadata']['link']
#     # print(url)
#     try:
#         articles[i] = scrape_selenium_headless(url,browser='undetected_chrome')
#     except:
#         pass
# indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
# print(len(indices2))

grabbing text from url: 100%|██████████| 821/821 [02:29<00:00,  5.49it/s]

199


In [6]:
indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
print(len(indices2))
articles_not_in_indices2 = [article for i, article in enumerate(articles) if i not in indices2]

199


## GLiNER featurize

In [220]:
from DOTS.feat import gliner_feat

ImportError: cannot import name 'gliner_feat' from 'DOTS.feat' (/Users/apple/WRK/dcolinmorgan/dots/DOTS/feat.py)

In [148]:
from gliner import GLiNER
model = GLiNER.from_pretrained("urchade/gliner_base")

indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
print(len(indices2))
text = [article for i, article in enumerate(articles) if i not in indices2]
hh = [hit for i, hit in enumerate(hits) if i not in indices2]

199


In [181]:
[len(hh),len(text)]

[622, 622]

In [186]:
%%time
import pandas as pd
# labels https://github.com/graphistry/graphistrygpt/blob/4b761af233c52109fb6bca2aec4e54d8c0ffa4ad/graphistrygpt/plugins/dt/models.py#L108
# labels=['disaster', 'flood', 'weather', 'hurricane', 'fire', 'date', 'location']#, 'politician', 'corporation', 'government_agency', 'military']
labels=[ "avalanche",
        "biological","blizzard",
        "chemical","contamination","cyber",
        "drought",
        "earthquake","explosion",
        "fire","flood",
        "heat","hurricane",
        "infrastructure",
        "landslide",
        "nuclear",
        "pandemic","power",
        "radiological","riot",
        "sabotage",
        "terror","tornado","transport","tsunami",
        "volcano"]
df = pd.DataFrame(columns=['Title','URL', 'Text', 'Label', 'Date','Location'])

for hit,article in zip(hh[:100],text[:100]):
    entities = model.predict_entities(''.join(article), labels)
    for entity in entities:
        row = pd.DataFrame({'Title': [hit['_source']['metadata']['title']],
                            'URL': [hit['_source']['metadata']['link']], 
                            'Text': [entity["text"]], 
                            'Label': [entity["label"]]})
        df = pd.concat([df, row], ignore_index=True)
print(len(df))

252


In [187]:
df2 = df.groupby(['Title','URL','Label']).agg({'Text': ' '.join}).reset_index()

In [196]:
df_pivot.columns[2:]

Index(['biological', 'cyber', 'earthquake', 'explosion', 'fire', 'flood',
       'heat', 'infrastructure', 'landslide', 'power', 'riot', 'terror',
       'tornado', 'transport'],
      dtype='object', name='Label')

In [197]:
df_pivot = df2.pivot(index=['Title','URL'], columns='Label', values='Text')
df_pivot.reset_index(inplace=True)
df_pivot = df_pivot.dropna(subset=df_pivot.columns[2:],how='all')
df_pivot

Label,Title,URL,biological,cyber,earthquake,explosion,fire,flood,heat,infrastructure,landslide,power,riot,terror,tornado,transport
0,"1 dead, 2 children wounded after 13 y/o goes o...",https://news.google.com/rss/articles/CBMiX2h0d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mass shooting mass shooting,NaN,ambulances
1,"1 dead, 2 injured in fire at northwest Indiana...",https://news.google.com/rss/articles/CBMiogFod...,NaN,browser,NaN,NaN,fire,NaN,NaN,Mount Zion Suburban Apartments,NaN,NaN,NaN,NaN,NaN,NaN
2,10th suspect held in Moscow terror attack - Th...,https://news.google.com/rss/articles/CBMieWh0d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,massacre,NaN,NaN,NaN
3,2 people rescued from Humber River following l...,https://news.google.com/rss/articles/CBMibmh0d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Humber River culverts,landslide landslide landslide,NaN,NaN,NaN,NaN,truck car
4,22 terrorists arrested during 232 IBOs conduct...,https://news.google.com/rss/articles/CBMiYWh0d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Al-Qaeda,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Unexploded WW2 bomb detonated on Jersey's east...,https://news.google.com/rss/articles/CBMiOmh0d...,NaN,NaN,NaN,controlled explosion Guatemala landfill fire,Russian strikes Guatemala landfill fire,NaN,NaN,Seymour Tower Eiffel Tower,NaN,NaN,NaN,NaN,NaN,NaN
65,Utah Man Arrested for Assaulting Officers Duri...,https://news.google.com/rss/articles/CBMieWh0d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,civil disorder,NaN,NaN,NaN
66,Video shows flames shooting from Philadelphia ...,https://news.google.com/rss/articles/CBMifmh0d...,NaN,NaN,NaN,NaN,fire fire fire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,"Video: Storm expected to bring heavy snow, gus...",https://news.google.com/rss/articles/CBMiUGh0d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,power outages,NaN,NaN,NaN,NaN


In [198]:

from collections import Counter

# Your list
lst = df.Text[df.Label=='biological']

# Count the frequency of each element
counter = Counter(lst)

# Sort the list by frequency
sorted_lst = sorted(lst, key=lambda x: -counter[x])

print(pd.unique(sorted_lst))

['spring Chinook' 'smolts']


In [199]:

from collections import Counter

# Your list
lst = df.Text[df.Label=='earthquake']

# Count the frequency of each element
counter = Counter(lst)

# Sort the list by frequency
sorted_lst = sorted(lst, key=lambda x: -counter[x])

print(pd.unique(sorted_lst))

['3.7 magnitude earthquake' '3.3 quake' '2.3-magnitude earthquake']


In [200]:

from collections import Counter

# Your list
lst = df.Text[df.Label=='fire']

# Count the frequency of each element
counter = Counter(lst)

# Sort the list by frequency
sorted_lst = sorted(lst, key=lambda x: -counter[x])

print(pd.unique(sorted_lst))

['fire' 'six-alarm fire' 'Russian strikes' 'Guatemala landfill fire'
 'Israeli fire' 'Fire' 'flames' 'Blaze' 'truck fire' 'multi-alarm fire'
 'brush fire']


In [201]:
import pandas as pd
import os
os.getcwd()
import graphistry

from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
g_pass = os.getenv('g_pass')
g_user = os.getenv('g_user')

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username=g_user, password=g_pass) ## key id, secret key
from graphistry.features import search_model, topic_model, ngrams_model, ModelDict, default_featurize_parameters, default_umap_parameters

In [203]:
%%time
g=graphistry.nodes(df_pivot.drop(['URL','Title'],axis=1))
g2 = g.umap()#df_pivot.drop(['URL','Title'],axis=1),**topic_model)
g2 = g2.dbscan() #min_dist=1, min_samples=3)
# g3 = g2.transform_dbscan(df_pivot.drop(['URL','Title'],axis=1),return_graph=False)
df2=pd.DataFrame(g2.get_matrix())

max_index_per_row = df2.idxmax(axis=1)
top_3_indices = max_index_per_row.value_counts().index[:10]
top_3_indices

* Ignoring target column of shape (69, 0) in UMAP fit, as it is not one dimensional

CPU times: user 1.13 s, sys: 244 ms, total: 1.37 s
Wall time: 1.16 s


Index(['biological_nan', 'cyber_nan'], dtype='object')

In [204]:
# import seaborn as sns
# import matplotlib.colors as mcolors
# palette = sns.color_palette("hls", 10)
# hex_palette = [mcolors.rgb2hex(color) for color in palette]

In [205]:
g2.plot() # encode_point_color('Label', palette=hex_palette, as_continuous=False).plot()

## gpy.feat

In [ ]:
import pandas as pd
import os
os.getcwd()
import graphistry

from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
g_pass = os.getenv('g_pass')
g_user = os.getenv('g_user')

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username=g_user, password=g_pass) ## key id, secret key
from graphistry.features import search_model, topic_model, ngrams_model, ModelDict, default_featurize_parameters, default_umap_parameters

In [207]:
indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
(len(indices2))
articles_not_in_indices2 = [article for i, article in enumerate(articles) if i not in indices2]

df=pd.Series(articles_not_in_indices2).to_frame()
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].astype(str)
df.columns=['text']
df['text'] = df['text'].str.replace("', '", ' ')
df['text'] = df['text'].str.replace('", "', ' ')
df['text'] = df['text'].str.replace('" "', ' ')
df['text'] = df['text'].str.replace("' '", ' ')
df['text'] = df['text'].str.replace("\'", ' ')
df['text'] = df['text'].str.replace("]", ' ')
df['text'] = df['text'].str.replace("[", ' ')
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].astype(str)

In [208]:
df

,text
0,The Nigerian Army has confirmed a minor expl...
1,"Raipur, Apr 2 (PTI) In a major anti-insurge..."
2,Notifications can be managed in browser pref...
3,The police arrested a 19-year-old Rotterdam ...
4,"""A major Russian attack on energy infrastruct..."
...,...
617,Police detain protesters during the so-calle...
618,"The latest breaking updates, delivered strai..."
619,"WEATHER ALERT Christian Terry, Digital Conte..."
620,"LINCOLN, Neb. (KOLN) - A man was cited after..."


In [209]:
%%time
g=graphistry.nodes(df)
g2 = g.umap(df,**topic_model)
g2 = g2.dbscan() #min_dist=1, min_samples=3)
g3 = g2.transform_dbscan(df,return_graph=False)
df2=pd.DataFrame(g2.get_matrix())

max_index_per_row = df2.idxmax(axis=1)
top_3_indices = max_index_per_row.value_counts().index[:10]
top_3_indices

* Ignoring target column of shape (622, 0) in UMAP fit, as it is not one dimensional

CPU times: user 17min 14s, sys: 1min 50s, total: 19min 4s
Wall time: 3min 3s


Index(['text: firefighters, firefighter, alcofirefighters',
       'text: ljlo9b5ncj, h9zep9aikz, yvmqjhizmn',
       'text: newfoundland, sudbury, kutzner',
       'text: rewritten, boyfriend, physically',
       'text: northeasterly, northwestern, northeastern',
       'text: ___________________________________________________________________________________________________________, 87301f6d7b5384de, 873021da98530510',
       'text: spokesperson, 01676637, services',
       'text: subscribe, subscriber, geopolitics',
       'text: gkj0hjujdr, meteorologist, qmmu4tcxt1',
       'text: menafn02042024000045015687id1108047153, menafn02042024000063011010id1108048574, menafn02042024000045015687id1108047138'],
      dtype='object')

In [210]:
g2.plot()

## distilroberta featurize

In [212]:
from DOTS.feat import featurize_stories
import graphistry

In [213]:
featurize_stories(str(articles[211]), top_k = 3, max_len=512)

['severe weather', 'falling rain', 'kindergarten']

In [214]:
rank_articles=[]
from tqdm import tqdm
indices3 = [i for i in range(len(articles)) if i not in indices2]

for i in tqdm(indices3):
    try:
        cc = featurize_stories(str(articles[i]), top_k = 3, max_len=512)
        rank_articles.append([cc])
    except:
        pass

100%|██████████| 622/622 [16:14<00:00,  1.57s/it]


In [215]:
flattened_list = [item for sublist in rank_articles for item in sublist]

In [216]:
import pandas as pd
import os
os.getcwd()
import graphistry

from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
g_pass = os.getenv('g_pass')
g_user = os.getenv('g_user')

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username=g_user, password=g_pass) ## key id, secret key

In [217]:
# pd.DataFrame(flattened_list).to_csv('big_test.txt')
# data=pd.read_csv('../../../big_test.txt',index_col=0)

In [218]:
# import pandas as pd
data=pd.DataFrame(flattened_list)  # each ranked feature is a row
data=data.dropna(how='any')
data=data.drop_duplicates()

object_columns = data.select_dtypes(include=['object']).columns
data[object_columns] = data[object_columns].astype(str)

# g = graphistry.edges(data).materialize_nodes()
g = graphistry.nodes(data)
# g2 = g.featurize()
# g2._clustersummary()

g22 = g.umap()
g22=g22.dbscan()#min_dist=0.1, min_samples=3) #82 groups
g33 = g22.transform_dbscan(data,return_graph=False)
df22=pd.DataFrame(g22.get_matrix())

max_index_per_row = df22.idxmax(axis=1)
top_33_indices = max_index_per_row.value_counts().index[:10]
top_33_indices

* Ignoring target column of shape (539, 0) in UMAP fit, as it is not one dimensional

Index(['2: weather, severe, death', '2: thunderstorms, storms, widespread',
       '2: belotserkovsky, impoverishment, urmărește',
       '2: youssef, hussein, county', '0: firefighters, night, fires',
       '2: dahdouh, pourahmadi, damage',
       '2: firefighters, firefighter, daughters',
       '2: afternoon, aftershocks, enter',
       '2: authorities, mississippi, puppies',
       '1: israeli, warplanes, rezidențiale'],
      dtype='object')

In [219]:
g22.plot() # .encode_point_color('_dbscan',palette=["hotpink", "dodgerblue"],as_continuous=True).plot()